In [3]:
# add for tensorboard code
from __future__ import absolute_import, division, print_function, unicode_literals
import argparse
import json
from tensorflow.python.lib.io import file_io

# origin fairing code
import tensorflow as tf
import os

import requests

class MyFashionMnist(object):
  def train(self):
           
    def download(url: str, dest_folder: str):
      if not os.path.exists(dest_folder):
        os.makedirs(dest_folder)  # create folder if it does not exist
 
      filename = url.split('/')[-1].replace(" ", "_")  # be careful with file names
      file_path = os.path.join(dest_folder, filename)
 
      r = requests.get(url, stream=True)
      if r.ok:
        print("saving to", os.path.abspath(file_path))
        with open(file_path, 'wb') as f:
          for chunk in r.iter_content(chunk_size=1024 * 8):
            if chunk:
              f.write(chunk)
              f.flush()
              os.fsync(f.fileno())
      else:  # HTTP status code 4XX/5XX
        print("Download failed: status code {}\n{}".format(r.status_code, r.text))
        
    download("http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz", dest_folder="/mnt/data/")
    download("http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz", dest_folder="/mnt/data/")
    download("http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz", dest_folder="/mnt/data/")
    download("http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz", dest_folder="/mnt/data/")

if __name__ == '__main__':
    if os.getenv('FAIRING_RUNTIME', None) is None:
        from kubeflow import fairing
        from kubeflow.fairing.kubernetes import utils as k8s_utils
        
        DOCKER_REGISTRY = 'kubeflow-registry.default.svc.cluster.local:30000'   # 프라이빗 레지스트리

        fairing.config.set_builder(
            'append',
            image_name='datadownload-job', # here 
            base_image='brightfly/kubeflow-jupyter-lab:tf2.0-cpu',
            registry=DOCKER_REGISTRY, 
            push=True
        )
        # cpu 1, memory 5GiB
        fairing.config.set_deployer('job',
                                    namespace='admin', # here
                                    pod_spec_mutators=[
                                        k8s_utils.get_resource_mutator(cpu=1,  # here
                                                                       memory=5)]
                                   )
        fairing.config.run()
    else:
        remote_train = MyFashionMnist()
        remote_train.train()

[I 201027 11:28:12 config:123] Using preprocessor: <kubeflow.fairing.preprocessors.converted_notebook.ConvertNotebookPreprocessor object at 0x7fa9abf60588>
[I 201027 11:28:12 config:125] Using builder: <kubeflow.fairing.builders.append.append.AppendBuilder object at 0x7fa93008e240>
[I 201027 11:28:12 config:127] Using deployer: <kubeflow.fairing.builders.append.append.AppendBuilder object at 0x7fa93008e240>
[W 201027 11:28:12 append:50] Building image using Append builder...
[I 201027 11:28:12 base:105] Creating docker context: /tmp/fairing_context_ys7rimw_
[I 201027 11:28:12 converted_notebook:127] Converting 01-data-download.ipynb to 01-data-download.py
[I 201027 11:28:12 docker_creds_:234] Loading Docker credentials for repository 'brightfly/kubeflow-jupyter-lab:tf2.0-cpu'
[W 201027 11:28:14 append:54] Image successfully built in 2.3333871900103986s.
[W 201027 11:28:14 append:94] Pushing image kubeflow-registry.default.svc.cluster.local:30000/datadownload-job:9A1FC451...
[I 201027 1

saving to /mnt/data/train-images-idx3-ubyte.gz
saving to /mnt/data/train-labels-idx1-ubyte.gz
saving to /mnt/data/t10k-labels-idx1-ubyte.gz
saving to /mnt/data/t10k-images-idx3-ubyte.gz


[W 201027 11:28:28 job:162] Cleaning up job fairing-job-l47tg...
